In [1]:
import os
import cv2
import random
import numpy as np
from PIL import Image, ImageEnhance, ImageOps, ImageFilter
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import shutil

# 设置随机种子
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

workshop_dir = os.path.abspath("..")  
original_dir = os.path.join(workshop_dir, "dataprocess", "original")
bad_samples_dir = os.path.join(workshop_dir, "dataprocess", "bad_samples")

# 增强后数据分为 train 和 test 两个目录
aug_train_dir = os.path.join(workshop_dir, "dataprocess", "augmented", "train")
aug_test_dir = os.path.join(workshop_dir, "dataprocess", "augmented", "test")

# 清空并重新创建目录
for d in [aug_train_dir, aug_test_dir]:
    if os.path.exists(d):
        shutil.rmtree(d)
    os.makedirs(d, exist_ok=True)

print("当前工作目录:", workshop_dir)
print("原始数据目录:", original_dir)
print("Bad Samples 数据目录:", bad_samples_dir)
print("增强后训练集保存目录:", aug_train_dir)
print("增强后测试集保存目录:", aug_test_dir)

当前工作目录: e:\1.code\Jupyter-notebook\MUST-DataScience\1-groupwork
原始数据目录: e:\1.code\Jupyter-notebook\MUST-DataScience\1-groupwork\dataprocess\original
Bad Samples 数据目录: e:\1.code\Jupyter-notebook\MUST-DataScience\1-groupwork\dataprocess\bad_samples
增强后训练集保存目录: e:\1.code\Jupyter-notebook\MUST-DataScience\1-groupwork\dataprocess\augmented\train
增强后测试集保存目录: e:\1.code\Jupyter-notebook\MUST-DataScience\1-groupwork\dataprocess\augmented\test


In [2]:
TARGET_SIZE = (224, 224)  # MobileNet / ResNet / EfficientNet 通用尺寸

def preprocess_image(img_path):
    """使用 PIL 读取图片，统一尺寸，去噪，颜色校正，归一化"""
    try:
        # PIL 读取图片
        img = Image.open(img_path).convert("RGB")
        img = img.resize(TARGET_SIZE, Image.Resampling.BILINEAR)
        img = np.array(img)
        #  归一化到 [0,1]
        img = img / 255.0

        return img
    except Exception as e:
        print("读取失败:", img_path, e)
        return None


In [3]:
def random_geometric(img):
    """旋转 ±15° + 翻转 + 随机裁剪"""
    img = Image.fromarray((img*255).astype(np.uint8))
    
    # 旋转
    angle = np.random.uniform(-15, 15)
    img = img.rotate(angle,resample=Image.Resampling.BILINEAR)
    
    # 水平镜像翻转
    if np.random.rand() < 0.5:
        img = ImageOps.mirror(img)
    
    
    # 随机裁剪
    w, h = img.size
    scale = np.random.uniform(0.85, 1.0)
    crop_w, crop_h = int(w*scale), int(h*scale)
    x = np.random.randint(0, w - crop_w + 1)
    y = np.random.randint(0, h - crop_h + 1)
    img = img.crop((x, y, x+crop_w, y+crop_h))
    img = img.resize(TARGET_SIZE,Image.Resampling.BILINEAR)
    
    return np.array(img)/255.0

def augment_image(img):
    """组合增强 pipeline"""
    img = random_geometric(img)
    return img

def save_image_cv2(filepath,img_array):
    """
    辅助保存函数：处理 RGB -> BGR 的转换和 uint8 转换
    """
    # 1. 还原到 [0, 255]
    img_uint8 = (img_array * 255).astype(np.uint8)
    # 2. 颜色空间转换 (RGB -> BGR)，因为 cv2.imwrite 需要 BGR
    img_bgr = cv2.cvtColor(img_uint8, cv2.COLOR_RGB2BGR)
    # 3. 保存
    cv2.imwrite(filepath, img_bgr)

In [ ]:
# ========== 配置参数 ==========
DEFAULT_AUG_TIMES = 30  # 默认增强次数
TEST_RATIO = 0.30  # 30% 作为测试集

# 特殊类别的增强次数设置
SPECIAL_AUG_TIMES = {
    "Pearl_millet(bajra)": 25,  # 该类别有39张，设置25次增强
}

# ========== 第1步：统计每个类别的原始图片数量 ==========
print("=" * 60)
print("第1步：统计各类别原始图片数量")
print("=" * 60)

# 获取所有类别
classes = set()
for source_dir in [original_dir, bad_samples_dir]:
    if os.path.exists(source_dir):
        for d in os.listdir(source_dir):
            if os.path.isdir(os.path.join(source_dir, d)):
                classes.add(d)
classes = sorted(list(classes))
print(f"共有 {len(classes)} 个类别: {classes}")

# 统计每个类别的图片
class_images = {}  # {类别: [图片路径列表]}

for cls in classes:
    class_images[cls] = []
    
    # 从 original 目录收集
    orig_cls_dir = os.path.join(original_dir, cls)
    if os.path.exists(orig_cls_dir):
        for fname in os.listdir(orig_cls_dir):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                class_images[cls].append(os.path.join(orig_cls_dir, fname))
    
    # 从 bad_samples 目录收集
    bad_cls_dir = os.path.join(bad_samples_dir, cls)
    if os.path.exists(bad_cls_dir):
        for fname in os.listdir(bad_cls_dir):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                class_images[cls].append(os.path.join(bad_cls_dir, fname))

# 打印统计信息
print("\n各类别原始图片数量：")
for cls in classes:
    print(f"  {cls}: {len(class_images[cls])} 张")

# ========== 第2步：设置每个类别的增强次数 ==========
print("\n" + "=" * 60)
print("第2步：设置各类别增强次数")
print("=" * 60)

class_aug_times = {}
for cls in classes:
    if cls in SPECIAL_AUG_TIMES:
        class_aug_times[cls] = SPECIAL_AUG_TIMES[cls]
    else:
        class_aug_times[cls] = DEFAULT_AUG_TIMES

print("\n各类别增强策略：")
for cls in classes:
    num_orig = len(class_images[cls])
    aug_times = class_aug_times[cls]
    total = num_orig * (1 + aug_times) if num_orig > 0 else 0
    special_mark = " ★" if cls in SPECIAL_AUG_TIMES else ""
    print(f"  {cls}: {num_orig} 张原图 × (1 + {aug_times}次增强) = {total} 张{special_mark}")

# ========== 第3步：处理每个类别 ==========
print("\n" + "=" * 60)
print("第3步：数据增强并划分训练/测试集")
print("=" * 60)

total_train = 0
total_test = 0

for cls in classes:
    images = class_images[cls]
    aug_times = class_aug_times[cls]
    
    if len(images) == 0:
        print(f"\n跳过类别 {cls}：无图片")
        continue
    
    # 创建输出目录
    os.makedirs(os.path.join(aug_train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(aug_test_dir, cls), exist_ok=True)
    
    # 随机打乱图片顺序
    random.shuffle(images)
    
    # 划分：30% 的原始图片用于生成测试集
    test_split = int(len(images) * TEST_RATIO)
    test_images = images[:test_split]
    train_images = images[test_split:]
    
    print(f"\n处理类别: {cls}")
    print(f"  原图: {len(images)} 张 → 训练: {len(train_images)} 张, 测试: {len(test_images)} 张")
    
    # 处理训练集图片
    train_count = 0
    for fpath in tqdm(train_images, desc=f"{cls} [TRAIN]"):
        img = preprocess_image(fpath)
        if img is None:
            continue
        
        base = os.path.splitext(os.path.basename(fpath))[0]
        
        # 保存原图
        save_image_cv2(os.path.join(aug_train_dir, cls, f"{base}_base.jpg"), img)
        train_count += 1
        
        # 数据增强
        for i in range(aug_times):
            aug_img = augment_image(img)
            save_image_cv2(os.path.join(aug_train_dir, cls, f"{base}_aug{i:02d}.jpg"), aug_img)
            train_count += 1
    
    # 处理测试集图片
    test_count = 0
    for fpath in tqdm(test_images, desc=f"{cls} [TEST]"):
        img = preprocess_image(fpath)
        if img is None:
            continue
        
        base = os.path.splitext(os.path.basename(fpath))[0]
        
        # 保存原图
        save_image_cv2(os.path.join(aug_test_dir, cls, f"{base}_base.jpg"), img)
        test_count += 1
        
        # 数据增强（测试集也做增强以保持一致性）
        for i in range(aug_times):
            aug_img = augment_image(img)
            save_image_cv2(os.path.join(aug_test_dir, cls, f"{base}_aug{i:02d}.jpg"), aug_img)
            test_count += 1
    
    total_train += train_count
    total_test += test_count
    print(f"  生成: 训练集 {train_count} 张, 测试集 {test_count} 张")

# ========== 最终统计 ==========
print("\n" + "=" * 60)
print("数据增强完成！最终统计：")
print("=" * 60)
print(f"训练集总数: {total_train} 张")
print(f"测试集总数: {total_test} 张")
print(f"总计: {total_train + total_test} 张")

print("\n各类别最终样本数：")
for cls in classes:
    train_cls_dir = os.path.join(aug_train_dir, cls)
    test_cls_dir = os.path.join(aug_test_dir, cls)
    train_n = len(os.listdir(train_cls_dir)) if os.path.exists(train_cls_dir) else 0
    test_n = len(os.listdir(test_cls_dir)) if os.path.exists(test_cls_dir) else 0
    print(f"  {cls}: 训练 {train_n} + 测试 {test_n} = {train_n + test_n}")

第1步：统计各类别原始图片数量
共有 8 个类别: ['Cherry', 'Cucumber', 'Pearl_millet(bajra)', 'Tobacco-plant', 'banana', 'cotton', 'maize', 'wheat']

各类别原始图片数量：
  Cherry: 32 张
  Cucumber: 31 张
  Pearl_millet(bajra): 39 张
  Tobacco-plant: 33 张
  banana: 31 张
  cotton: 32 张
  maize: 31 张
  wheat: 31 张

第2步：设置各类别增强次数

各类别增强策略：
  Cherry: 32 张原图 × (1 + 30次增强) = 992 张
  Cucumber: 31 张原图 × (1 + 30次增强) = 961 张
  Pearl_millet(bajra): 39 张原图 × (1 + 25次增强) = 1014 张 ★
  Tobacco-plant: 33 张原图 × (1 + 30次增强) = 1023 张
  banana: 31 张原图 × (1 + 30次增强) = 961 张
  cotton: 32 张原图 × (1 + 30次增强) = 992 张
  maize: 31 张原图 × (1 + 30次增强) = 961 张
  wheat: 31 张原图 × (1 + 30次增强) = 961 张

第3步：数据增强并划分训练/测试集

处理类别: Cherry
  原图: 32 张 → 训练: 23 张, 测试: 9 张


Cherry [TEST]: 100%|██████████| 9/9 [00:01<00:00,  8.34it/s]


  生成: 训练集 713 张, 测试集 279 张

处理类别: Cucumber
  原图: 31 张 → 训练: 22 张, 测试: 9 张


Cucumber [TEST]: 100%|██████████| 9/9 [00:01<00:00,  7.73it/s]


  生成: 训练集 682 张, 测试集 279 张

处理类别: Pearl_millet(bajra)
  原图: 39 张 → 训练: 28 张, 测试: 11 张


...

  生成: 训练集 713 张, 测试集 279 张

处理类别: maize
  原图: 31 张 → 训练: 22 张, 测试: 9 张


maize [TEST]: 100%|██████████| 9/9 [00:01<00:00,  8.11it/s]


  生成: 训练集 682 张, 测试集 279 张

处理类别: wheat
  原图: 31 张 → 训练: 22 张, 测试: 9 张


wheat [TEST]: 100%|██████████| 9/9 [00:01<00:00,  8.16it/s]

  生成: 训练集 682 张, 测试集 279 张

数据增强完成！最终统计：
训练集总数: 5626 张
测试集总数: 2239 张
总计: 7865 张

各类别最终样本数：
  Cherry: 训练 713 + 测试 279 = 992
  Cucumber: 训练 651 + 测试 279 = 930
  Pearl_millet(bajra): 训练 676 + 测试 260 = 936
  Tobacco-plant: 训练 744 + 测试 279 = 1023
  banana: 训练 620 + 测试 279 = 899
  cotton: 训练 651 + 测试 248 = 899
  maize: 训练 558 + 测试 217 = 775
  wheat: 训练 682 + 测试 279 = 961
